In [6]:
import sys
!{sys.executable} -m pip install pandas numpy scikit-learn

import pandas as pd
import numpy as np
import re

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

import pickle
from datetime import datetime
import time


true=True
false=False

DTYPE_DICT = {"INCIDENT_TYPE_DESC":"uint16","UNITS_ONSCENE":"uint8","TOTAL_INCIDENT_DURATION":"uint16","ACTION_TAKEN1_DESC":"int16","ACTION_TAKEN2_DESC":"int16","ACTION_TAKEN3_DESC":"int16","PROPERTY_USE_DESC":"int16","BOROUGH_DESC":"uint8","SEASON":"uint8","TIME_OF_DAY":"uint8","WEEKDAY":"uint8"}
DROP_COLUMNS = ["IM_INCIDENT_KEY","FIRE_BOX", "ARRIVAL_DATE_TIME", "LAST_UNIT_CLEARED_DATE_TIME", "HIGHEST_LEVEL_DESC", "STREET_HIGHWAY", "ZIP_CODE", "FLOOR", "CO_DETECTOR_PRESENT_DESC", "FIRE_ORIGIN_BELOW_GRADE_FLAG", "STORY_FIRE_ORIGIN_COUNT", "FIRE_SPREAD_DESC", "DETECTOR_PRESENCE_DESC", "AES_PRESENCE_DESC", "STANDPIPE_SYS_PRESENT_FLAG"]

Import the data from the split CSV files; in doing so, filter out all rows which do not have one of our input variables, and drop all columns we're not using.

In [7]:
def pud_filt(row):
    x = row["PROPERTY_USE_DESC"].split(" ")[0]
    if x == "NNN":
        return -1
    if x == "UUU":
        return 999
    return x

def itd_filt(row):
    # print(row)
    return re.sub('[^0-9]','', row["INCIDENT_TYPE_DESC"].split(" ")[0])

def idt_filt(row):
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p')
    return time.mktime(s.timetuple())

def extract_season(row):
    # 0 = Winter (Dec-Feb)
    # 1 = Spring (Mar-May)
    # 2 = Summer (June-Aug)
    # 3 = Fall (Sep-Nov)
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timetuple()
    if s.tm_mon < 3 or s.tm_mon == 12:
        return 0
    elif s.tm_mon < 6:
        return 1
    elif s.tm_mon < 9:
        return 2
    else:
        return 3

def extract_tod(row):
    # 0 = midnight-6am
    # 1 = 6am-noon
    # 2 = noon-6pm
    # 3 = 6pm-midnight
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timetuple()
    if s.tm_hour < 6:
        return 0
    elif s.tm_hour < 12:
        return 1
    elif s.tm_hour < 18:
        return 2
    else:
        return 3

def extract_weekday(row):
    # 0 = monday, 6=sunday
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timetuple()
    return s.tm_wday



def extract_hour(row):
    # 0 - 24 hrs
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timetuple()
    return s.tm_hour

def extract_month(row):
    # 0 - 12
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timetuple()
    return s.tm_mon

def extract_year(row):
    # 2013-2018, i think
    # 2014-2017 just to be safe
    x = row["INCIDENT_DATE_TIME"]
    s = datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').timetuple()
    return s.tm_year


        

data = None
for i in range(3):
    print("Processing file {} of 3:".format(i+1),end="")
    x = pd.read_csv("NewCSVs/file{}.csv".format(i+1), low_memory=false)
    print(".",end="")
    x.drop(DROP_COLUMNS, axis=1, inplace=True)
    print(".",end="")
    x.dropna(axis=0, subset=["INCIDENT_TYPE_DESC", "INCIDENT_DATE_TIME", "BOROUGH_DESC", "PROPERTY_USE_DESC"], inplace=True)
    print(".",end="")
    x.fillna(0, inplace=True)
    print(".",end="")
    
    x["BOROUGH_DESC"] = x.apply(lambda row : int(row["BOROUGH_DESC"][0]), axis=1)
    print(".",end="")
    x["ACTION_TAKEN1_DESC"] = x.apply(lambda row : int(str(row["ACTION_TAKEN1_DESC"]).split(" ")[0]), axis=1)
    print(".",end="")
    x["ACTION_TAKEN2_DESC"] = x.apply(lambda row : int(str(row["ACTION_TAKEN2_DESC"]).split(" ")[0]) if str(row["ACTION_TAKEN2_DESC"]) != "nan" else -1, axis=1)
    print(".",end="")
    x["ACTION_TAKEN3_DESC"] = x.apply(lambda row : int(str(row["ACTION_TAKEN3_DESC"]).split(" ")[0]) if str(row["ACTION_TAKEN3_DESC"]) != "nan" else -1, axis=1)
    print(".",end="")
    x["INCIDENT_TYPE_DESC"] = x.apply(lambda row : itd_filt(row), axis=1)
    print(".",end="")
    x["PROPERTY_USE_DESC"] = x.apply(lambda row : pud_filt(row), axis=1)
    print(".",end="")
    x["SEASON"] = x.apply(lambda row : extract_season(row), axis=1)
    print(".",end="")
    x["TIME_OF_DAY"] = x.apply(lambda row : extract_tod(row), axis=1)
    print(".",end="")
    x["WEEKDAY"] = x.apply(lambda row : extract_weekday(row), axis=1)
    print(".",end="")

    x["HOUR"] = x.apply(lambda row : extract_hour(row), axis=1)
    print(".",end="")
    x["MONTH"] = x.apply(lambda row : extract_month(row), axis=1)
    print(".",end="")
    x["YEAR"] = x.apply(lambda row : extract_year(row), axis=1)
    print(".",end="")

    x.drop(["INCIDENT_DATE_TIME"], axis=1, inplace=True)
    print(".",end="")

    x = x.astype(DTYPE_DICT)
    print(".",end="")
    if data is None:
        data = x
    else:
        data = pd.concat([data,x])
    print("; Finished file {}".format(i+1))

data = data.astype(DTYPE_DICT)
print("Done.")
data.head()

Processing file 1 of 3:..................; Finished file 1
Processing file 2 of 3:..................; Finished file 2
Processing file 3 of 3:..................; Finished file 3
Done.


,INCIDENT_TYPE_DESC,UNITS_ONSCENE,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ACTION_TAKEN3_DESC,PROPERTY_USE_DESC,BOROUGH_DESC,SEASON,TIME_OF_DAY,WEEKDAY,HOUR,MONTH,YEAR
0,300,1,2355,0,0,0,999,4,0,0,2,0,1,2020
1,735,2,763,86,0,0,999,2,0,0,2,0,1,2020
2,735,2,1920,86,0,0,999,3,0,0,2,0,1,2020
3,555,1,1108,45,0,0,429,1,0,0,2,0,1,2020
4,651,2,743,0,0,0,999,2,0,0,2,0,1,2020


Save validation data to a CSV

In [8]:
data.to_csv("validate.csv.gzip",compression="gzip",index=False)

Load the data

In [9]:
data = pd.read_csv("validate.csv.gzip",compression="gzip",dtype=DTYPE_DICT)
data.head(10)

print("Drop extra columns")
data.drop(["HOUR"], axis=1, inplace=True)
data.drop(["MONTH"], axis=1, inplace=True)
data.drop(["YEAR"], axis=1, inplace=True)
data = data.astype(DTYPE_DICT)

Drop extra columns


Begin validation by using training models from 2013 to mid 2018 on 2020 and 2021 data

UOS

In [10]:
def predict(df, model_dict):
    res = []
    for i,row in df.iterrows():
        itype = int(row.INCIDENT_TYPE_DESC)
        pud = int(row.PROPERTY_USE_DESC)
        if not itype in model_dict.keys() or not pud in model_dict[itype].keys():
            # This combination was not in the training data! return 0
            res.append(0)
            continue
        df = pd.DataFrame(data=[list(row)], columns=df.columns)
        res.append(model_dict[itype][pud].predict(df[["BOROUGH_DESC","SEASON","TIME_OF_DAY","WEEKDAY"]])[0])
    return res

# -----------------------------UNITS ONSCENE---------------------------------------

uos_mdl_dict = {}
with open('uos_dict_bayesian.pickle','rb') as file:
    uos_mdl_dict = pickle.load(file)

print("Predicting UOS with new data")
res = predict(data, uos_mdl_dict)
mae1 = mean_absolute_error(data.UNITS_ONSCENE,res)
print("Bayesian UOS MAE:", mae1)

# -------------------------TOTAL INCIDENT DURATION---------------------------------

tid_mdl_dict = {}
with open('tid_dict_bay.pickle','rb') as file:
    tid_mdl_dict = pickle.load(file)

print("Predicting TID with new data")
res = predict(data, tid_mdl_dict)
mae2 = mean_absolute_error(data.TOTAL_INCIDENT_DURATION,res)
print("Bayesian TID MAE:", mae2)

# --------------------------------ACTION 1----------------------------------------

at1_mdl_dict = {}
with open('at1_dict_ridge.pickle','rb') as file:
    at1_mdl_dict = pickle.load(file)

print("Predicting AT1 with new data")
res = predict(data, at1_mdl_dict)
mae3 = accuracy_score(data.ACTION_TAKEN1_DESC,res)
print("Ridge AT1 Accuracy:", mae3)

# --------------------------------ACTION 2----------------------------------------

at2_mdl_dict = {}
with open('at2_dict_ridge.pickle','rb') as file:
    at2_mdl_dict = pickle.load(file)

print("Predicting AT2 with new data")
res = predict(data, at2_mdl_dict)
mae4 = accuracy_score(data.ACTION_TAKEN2_DESC,res)
print("Ridge AT2 Accuracy:", mae4)

# --------------------------------ACTION 3----------------------------------------

at3_mdl_dict = {}
with open('at3_dict_ridge.pickle','rb') as file:
    at3_mdl_dict = pickle.load(file)

print("Predicting AT3 with new data")
res = predict(data, at3_mdl_dict)
mae5 = accuracy_score(data.ACTION_TAKEN3_DESC,res)
print("Ridge AT3 Accuracy:", mae5)



Predicting UOS with new data
Bayesian UOS MAE: 0.5933835770008786
Predicting TID with new data
Bayesian TID MAE: 726.6111395841277
Predicting AT1 with new data
Ridge AT1 Accuracy: 0.8863529333362952
Predicting AT2 with new data
Ridge AT2 Accuracy: 0.8945798253992936
Predicting AT3 with new data
Ridge AT3 Accuracy: 0.9591154452762289
